In [1]:
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 6.7 MB/s eta 0:00:00


In [6]:
import pyodbc
import pandas as pd

# Informations de connexion SQL Server
server = '127.0.0.1'  # Adresse de votre serveur SQL Server
database = 'TitanicDB'  # Nom de la base de données
table_name = 'TitanicData'  # Nom de la table cible
trusted_connection = True  # Utiliser l'authentification Windows

# Charger le fichier CSV
csv_file = 'titanic.csv'  # Chemin vers le fichier CSV
df = pd.read_csv(csv_file)

# Fonction de connexion à SQL Server
def connect_to_sql():
    try:
        if trusted_connection:
            conn = pyodbc.connect(
                f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
            )
        else:
            username = 'root'
            password = ''
            conn = pyodbc.connect(
                f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};"
            )
        print("Connexion réussie à SQL Server!")
        return conn
    except Exception as e:
        print(f"Erreur lors de la connexion à SQL Server : {e}")
        return None

# Création de la table dans SQL Server
def create_table(cursor):
    create_table_query = f'''
    CREATE TABLE {table_name} (
        PassengerId INT,
        Survived INT,
        Pclass INT,
        Name NVARCHAR(255),
        Sex NVARCHAR(50),
        Age FLOAT,
        SibSp INT,
        Parch INT,
        Ticket NVARCHAR(50),
        Fare FLOAT,
        Cabin NVARCHAR(50),
        Embarked NVARCHAR(50)
    )
    '''
    try:
        cursor.execute(create_table_query)
        print(f"Table '{table_name}' créée avec succès!")
    except Exception as e:
        print(f"Erreur lors de la création de la table : {e}")

# Insérer des données dans SQL Server
def insert_data(cursor, df):
    try:
        for index, row in df.iterrows():
            cursor.execute(f'''
                INSERT INTO {table_name} (PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', row['PassengerId'], row['Survived'], row['Pclass'], row['Name'], row['Sex'], row['Age'], row['SibSp'],
               row['Parch'], row['Ticket'], row['Fare'], row['Cabin'], row['Embarked'])
        print("Données insérées avec succès!")
    except Exception as e:
        print(f"Erreur lors de l'insertion des données : {e}")

# Main
conn = connect_to_sql()
if conn:
    cursor = conn.cursor()
    # Créer la table si nécessaire
    create_table(cursor)
    # Insérer les données du CSV
    insert_data(cursor, df)
    # Sauvegarder et fermer la connexion
    conn.commit()
    conn.close()
    print("Connexion fermée.")
